In [1]:
import pandas as pd
import numpy as np
import torch
from torch import cuda
from torch.utils.data import Dataset, DataLoader
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
from load_data import initialize_data
from reading_datasets import read_task
from labels_to_ids import labels_to_ids_mal
import time
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
def train(epoch, train_loader, model, optimizer, device, grad_step = 1, max_grad_norm = 10):
    tr_loss, tr_accuracy = 0, 0
    tr_precision, tr_recall = 0, 0
    tr_f1score = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    optimizer.zero_grad()
    
    for idx, batch in enumerate(train_loader):
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        if (idx + 1) % 20 == 0:
            print('FINSIHED BATCH:', idx, 'of', len(train_loader))

        #loss, tr_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
        output = model(input_ids=ids, attention_mask=mask, labels=labels)
        tr_loss += output[0]

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = output[1].view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
        
        # Compute Precision
        tmp_tr_precision = precision_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0 )
        tr_precision += tmp_tr_precision
        
        # Compute Recall
        tmp_tr_recall = recall_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0)
        tr_recall += tmp_tr_recall
        
        # Compute f1score
        tmp_tr_f1score = f1_score(labels.cpu().numpy(), predictions.cpu().numpy(), average= 'macro', zero_division=0)
        tr_f1score += tmp_tr_f1score
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=max_grad_norm
        )
        
        # backward pass
        output['loss'].backward()
        if (idx + 1) % grad_step == 0:
            optimizer.step()
            optimizer.zero_grad()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    tr_precision = tr_precision / nb_tr_steps
    tr_recall = tr_recall / nb_tr_steps
    tr_f1score= tr_f1score / nb_tr_steps
    #print(f"Training loss epoch: {epoch_loss}")
    #print(f"Training accuracy epoch: {tr_accuracy}")

    return model

In [3]:
def testing(model, testing_loader, labels_to_ids, device):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    eval_precision, eval_recall = 0, 0
    eval_f1score = 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
     
    
    ids_to_labels = dict((v,k) for k,v in labels_to_ids.items())

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)
            
            #loss, eval_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
            output = model(input_ids=ids, attention_mask=mask, labels=labels)

            eval_loss += output['loss'].item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = output[1].view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy
            
            # Compute Precision
            tmp_eval_precision = precision_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0)
            eval_precision += tmp_eval_precision
            
            # Compute Recall
            tmp_eval_recall = recall_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0)
            eval_recall += tmp_eval_recall
            
            # Compute f1score
            tmp_eval_f1score = f1_score(labels.cpu().numpy(), predictions.cpu().numpy(), average='macro', zero_division=0)
            eval_f1score += tmp_eval_f1score

    labels = [ids_to_labels[id.item()] for id in eval_labels]
    predictions = [ids_to_labels[id.item()] for id in eval_preds]
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    eval_precision = eval_precision / nb_eval_steps
    eval_recall = eval_recall / nb_eval_steps
    eval_f1score = eval_f1score / nb_eval_steps
    #print(f"Validation Loss: {eval_loss}")
    #print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions, eval_accuracy, eval_precision, eval_recall, eval_f1score

In [4]:
def main(n_epochs, model_name, model_save_flag, model_save_location, model_load_flag, model_load_location):
    #Initialization training parameters
    max_len = 256
    batch_size = 32
    grad_step = 1
    learning_rate = 1e-05
    initialization_input = (max_len, batch_size)

    #Reading datasets and initializing data loaders
    dataset_location = '../datasets/task_a/'

    train_data = read_task(dataset_location , split = 'mal_train_negative_augmented')
    dev_data = read_task(dataset_location , split = 'mal_sentiment_dev')
    #test_data = read_task(dataset_location , split = 'dev')#load test set
    labels_to_ids = labels_to_ids_mal
    #input_data = (train_data, dev_data, labels_to_ids)

    #Define tokenizer, model and optimizer
    device = 'cuda' if cuda.is_available() else 'cpu' #save the processing time
    if model_load_flag:
        tokenizer = AutoTokenizer.from_pretrained(model_load_location)
        model = AutoModelForSequenceClassification.from_pretrained(model_load_location)
    else: 
        tokenizer =  AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(labels_to_ids))
    optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)
    model.to(device)

    #Get dataloaders
    train_loader = initialize_data(tokenizer, initialization_input, train_data, labels_to_ids, shuffle = True)
    dev_loader = initialize_data(tokenizer, initialization_input, dev_data, labels_to_ids, shuffle = True)
    #test_loader = initialize_data(tokenizer, initialization_input, test_data, labels_to_ids, shuffle = True)#create test loader

    best_dev_acc = 0
    best_test_acc = 0
    best_dev_precision = 0
    best_test_precision = 0
    best_dev_recall = 0
    best_test_recall = 0
    best_dev_f1score = 0
    best_test_f1score = 0
    best_epoch = -1
    
    list_dev_acc = [] 
    list_test_acc = []  
    list_dev_precision = []  
    list_test_precision  = []  
    list_dev_recall = []  
    list_test_recall = []  
    list_dev_f1score = []  
    list_test_f1score = []
    
    for epoch in range(n_epochs):
        start = time.time()
        print(f"Training epoch: {epoch + 1}")

        #train model
        if not model_load_flag:
            model = train(epoch, train_loader, model, optimizer, device, grad_step)
        
        #testing and logging
        labels_dev, predictions_dev, dev_accuracy, dev_precision, dev_recall, dev_f1score = testing(model, dev_loader, labels_to_ids, device)
        print('DEV ACC:', dev_accuracy)
        print('DEV Precision:' , dev_precision)
        print('DEV Recall:' , dev_recall)
        print('DEV F1Score:' , dev_f1score)
        
        list_dev_acc.append(dev_accuracy)     
        list_dev_precision.append(dev_precision)   
        list_dev_recall.append(dev_recall)  
        list_dev_f1score.append(dev_f1score)  
        
        
        #labels_test, predictions_test, test_accuracy, test_precision, test_recall, test_f1score = testing(model, test_loader, labels_to_ids, device)
        #print('TEST ACC:', test_accuracy)
        #print('TEST Precision:' , test_precision)
        #print('TEST Recall:' , test_recall)
        #print('TEST F1Score:' , test_f1score)
        
        #list_test_acc.append(test_accuracy) 
        #list_test_precision.append(test_precision)  
        #list_test_recall.append(test_recall)
        #list_test_f1score.append(test_f1score) 

        #saving model
        if dev_accuracy > best_dev_acc:
            best_dev_acc = dev_accuracy
            #best_test_acc = test_accuracy
        if dev_precision > best_dev_precision:
            best_dev_precision = dev_precision
            #best_test_precision = test_precision
        if dev_recall > best_dev_recall:
            best_dev_recall = dev_recall
            #best_test_recall = test_recall
        if dev_f1score > best_dev_f1score:
            best_dev_f1score = dev_f1score
            #best_test_f1score = test_f1score
            best_epoch = epoch
            
            if model_save_flag:
                os.makedirs(model_save_location, exist_ok=True)
                tokenizer.save_pretrained(model_save_location)
                model.save_pretrained(model_save_location)

        now = time.time()
        print('BEST ACCURACY --> ', 'DEV:', round(best_dev_acc, 5))
        print('BEST PRECISION --> ', 'DEV:', round(best_dev_precision, 5))
        print('BEST RECALL --> ', 'DEV:', round(best_dev_recall, 5))
        print('BEST F1SCORE --> ', 'DEV:', round(best_dev_f1score, 5))
        print('TIME PER EPOCH:', (now-start)/60 )
        print()

    return best_dev_acc, best_test_acc, best_epoch, best_dev_precision, best_test_precision, best_dev_recall, best_test_recall, best_dev_f1score, best_test_f1score, list_dev_acc, list_test_acc, list_dev_precision, list_test_precision, list_dev_recall, list_test_recall, list_dev_f1score, list_test_f1score

In [5]:
if __name__ == '__main__':
    n_epochs = 10
    models = ['bert-base-multilingual-uncased']
    
    #model saving parameters
    model_save_flag = True
    model_load_flag = False
    
    overall_list_dev_acc = [] 
    overall_list_test_acc = []    
    overall_list_dev_precision = []  
    overall_list_test_precision  = []  
    overall_list_dev_recall = []  
    overall_list_test_recall = []  
    overall_list_dev_f1score = []  
    overall_list_test_f1score = [] 
    
    for i in range(5):
        
        for model_name in models:

            model_save_location = 'saved_models/' + model_name + 'Malayalam' + 'Augmented'+str(i)
            model_load_location = None

            best_dev_acc, best_test_acc, best_epoch, best_dev_precision, best_test_precision, best_dev_recall, best_test_recall, best_dev_f1score, best_test_f1score, list_dev_acc, list_test_acc, list_dev_precision, list_test_precision, list_dev_recall, list_test_recall, list_dev_f1score, list_test_f1score = main(n_epochs, model_name, model_save_flag, model_save_location, model_load_flag, model_load_location)
            
            overall_list_dev_acc.append(list_dev_acc) 
            overall_list_test_acc.append(list_test_acc) 
            overall_list_dev_precision.append(list_dev_precision)  
            overall_list_test_precision.append(list_test_precision) 
            overall_list_dev_recall.append(list_dev_recall)  
            overall_list_test_recall.append(list_test_recall)  
            overall_list_dev_f1score.append(list_dev_f1score)  
            overall_list_test_f1score.append(list_test_f1score) 

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/851k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.64M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/641M [00:00<?, ?B/s]

...

Training epoch: 1
FINSIHED BATCH: 19 of 632
FINSIHED BATCH: 39 of 632
FINSIHED BATCH: 59 of 632
FINSIHED BATCH: 79 of 632
FINSIHED BATCH: 99 of 632
FINSIHED BATCH: 119 of 632
FINSIHED BATCH: 199 of 632
FINSIHED BATCH: 219 of 632
FINSIHED BATCH: 239 of 632
FINSIHED BATCH: 259 of 632
FINSIHED BATCH: 279 of 632
FINSIHED BATCH: 299 of 632
FINSIHED BATCH: 319 of 632
FINSIHED BATCH: 339 of 632
FINSIHED BATCH: 359 of 632
FINSIHED BATCH: 379 of 632
FINSIHED BATCH: 399 of 632
FINSIHED BATCH: 419 of 632
FINSIHED BATCH: 439 of 632
FINSIHED BATCH: 459 of 632
FINSIHED BATCH: 479 of 632
FINSIHED BATCH: 499 of 632
FINSIHED BATCH: 519 of 632
FINSIHED BATCH: 539 of 632
FINSIHED BATCH: 559 of 632
FINSIHED BATCH: 579 of 632
FINSIHED BATCH: 599 of 632
FINSIHED BATCH: 619 of 632
Validation loss per 100 evaluation steps: 0.9042795300483704
DEV ACC: 0.6028645833333333
DEV Precision: 0.5240448057520152
DEV Recall: 0.5369401479614052
DEV F1Score: 0.4931755744839374
BEST ACCURACY -->  DEV: 0.60286
BEST PRECISIO

FINSIHED BATCH: 19 of 632
FINSIHED BATCH: 39 of 632
FINSIHED BATCH: 59 of 632
FINSIHED BATCH: 79 of 632
FINSIHED BATCH: 99 of 632
FINSIHED BATCH: 119 of 632
FINSIHED BATCH: 139 of 632
FINSIHED BATCH: 159 of 632
FINSIHED BATCH: 179 of 632
FINSIHED BATCH: 199 of 632
FINSIHED BATCH: 219 of 632
FINSIHED BATCH: 239 of 632
FINSIHED BATCH: 259 of 632
FINSIHED BATCH: 279 of 632
FINSIHED BATCH: 299 of 632
FINSIHED BATCH: 319 of 632
FINSIHED BATCH: 339 of 632
FINSIHED BATCH: 359 of 632
FINSIHED BATCH: 379 of 632
FINSIHED BATCH: 399 of 632
FINSIHED BATCH: 419 of 632
FINSIHED BATCH: 439 of 632
FINSIHED BATCH: 459 of 632
FINSIHED BATCH: 479 of 632
FINSIHED BATCH: 499 of 632
FINSIHED BATCH: 519 of 632
FINSIHED BATCH: 539 of 632
FINSIHED BATCH: 559 of 632
FINSIHED BATCH: 579 of 632
FINSIHED BATCH: 599 of 632
FINSIHED BATCH: 619 of 632
Validation loss per 100 evaluation steps: 1.0452138185501099
DEV ACC: 0.7198660714285714
DEV Precision: 0.648098535911727
DEV Recall: 0.6188124323884565
DEV F1Score: 0.

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

Training epoch: 1
FINSIHED BATCH: 19 of 632
FINSIHED BATCH: 39 of 632
FINSIHED BATCH: 59 of 632
FINSIHED BATCH: 79 of 632
FINSIHED BATCH: 99 of 632
FINSIHED BATCH: 119 of 632
FINSIHED BATCH: 139 of 632
FINSIHED BATCH: 159 of 632
FINSIHED BATCH: 179 of 632
FINSIHED BATCH: 199 of 632
FINSIHED BATCH: 219 of 632
FINSIHED BATCH: 239 of 632
FINSIHED BATCH: 259 of 632
FINSIHED BATCH: 279 of 632
FINSIHED BATCH: 299 of 632
FINSIHED BATCH: 319 of 632
FINSIHED BATCH: 339 of 632
FINSIHED BATCH: 359 of 632
FINSIHED BATCH: 379 of 632
FINSIHED BATCH: 399 of 632
FINSIHED BATCH: 419 of 632
FINSIHED BATCH: 439 of 632
FINSIHED BATCH: 459 of 632
FINSIHED BATCH: 539 of 632
FINSIHED BATCH: 559 of 632
FINSIHED BATCH: 579 of 632
FINSIHED BATCH: 599 of 632
FINSIHED BATCH: 619 of 632
Validation loss per 100 evaluation steps: 0.9990293979644775
DEV ACC: 0.6324404761904762
DEV Precision: 0.49540025238001784
DEV Recall: 0.5368601301243108
DEV F1Score: 0.4984768700163665
BEST ACCURACY -->  DEV: 0.63244
BEST PRECISI

FINSIHED BATCH: 19 of 632
FINSIHED BATCH: 39 of 632
FINSIHED BATCH: 59 of 632
FINSIHED BATCH: 79 of 632
FINSIHED BATCH: 99 of 632
FINSIHED BATCH: 119 of 632
FINSIHED BATCH: 139 of 632
FINSIHED BATCH: 159 of 632
FINSIHED BATCH: 179 of 632
FINSIHED BATCH: 199 of 632
FINSIHED BATCH: 219 of 632
FINSIHED BATCH: 239 of 632
FINSIHED BATCH: 259 of 632
FINSIHED BATCH: 279 of 632
FINSIHED BATCH: 299 of 632
FINSIHED BATCH: 319 of 632
FINSIHED BATCH: 339 of 632
FINSIHED BATCH: 359 of 632
FINSIHED BATCH: 379 of 632
FINSIHED BATCH: 399 of 632
FINSIHED BATCH: 419 of 632
FINSIHED BATCH: 439 of 632
FINSIHED BATCH: 459 of 632
FINSIHED BATCH: 479 of 632
FINSIHED BATCH: 499 of 632
FINSIHED BATCH: 519 of 632
FINSIHED BATCH: 539 of 632
FINSIHED BATCH: 559 of 632
FINSIHED BATCH: 579 of 632
FINSIHED BATCH: 599 of 632
FINSIHED BATCH: 619 of 632
Validation loss per 100 evaluation steps: 0.5934526324272156
DEV ACC: 0.7111235119047619
DEV Precision: 0.6546908067990413
DEV Recall: 0.6428394608863358
DEV F1Score: 0

In [6]:
print(best_dev_acc)

0.7243303571428571


In [7]:
print(best_epoch)

6


In [8]:
print(best_dev_precision)

0.6714171997700442


In [9]:
print(best_dev_recall)

0.6511672808040218


In [10]:
print(best_dev_f1score)

0.6363418407487581


In [11]:
print(overall_list_dev_acc)

[[0.6374627976190476, 0.7126116071428571, 0.7081473214285714, 0.7165178571428571, 0.7180059523809524, 0.7256324404761905, 0.7196800595238095, 0.7100074404761905, 0.7029389880952381, 0.7140997023809524], [0.6521577380952381, 0.6902901785714286, 0.6962425595238095, 0.7062872023809524, 0.7135416666666667, 0.7109375, 0.6941964285714286, 0.7027529761904762, 0.7005208333333333, 0.7224702380952381], [0.6028645833333333, 0.6886160714285714, 0.7040550595238095, 0.7027529761904762, 0.7109375, 0.6917782738095238, 0.712983630952381, 0.7198660714285714, 0.7111235119047619, 0.7124255952380952], [0.6558779761904762, 0.6860119047619048, 0.7044270833333333, 0.7077752976190476, 0.7172619047619048, 0.7081473214285714, 0.7213541666666667, 0.7105654761904762, 0.7200520833333333, 0.7120535714285714], [0.6324404761904762, 0.6765252976190476, 0.701078869047619, 0.7023809523809524, 0.7170758928571429, 0.7096354166666667, 0.7243303571428571, 0.7111235119047619, 0.7081473214285714, 0.7061011904761905]]


In [12]:
print(overall_list_dev_precision)

[[0.52126663187128, 0.618832306550856, 0.6145101342892698, 0.6227202764310238, 0.6676227243289657, 0.6653222164026635, 0.6553151217125602, 0.6523023907989769, 0.6315091057125829, 0.6572366416087047], [0.5249443523767316, 0.6035770616675086, 0.6247067863690071, 0.625667268509968, 0.6353898358355732, 0.6564242863750825, 0.6262390485193585, 0.6253345680223997, 0.6418205898480115, 0.665200324977249], [0.5240448057520152, 0.5901782966642266, 0.5958260297569418, 0.6239153426176937, 0.6321397664390256, 0.6162058627585495, 0.6344240844417757, 0.648098535911727, 0.6311704585987258, 0.6623279061501915], [0.5294776537745738, 0.6173147611317252, 0.645916974468293, 0.6579771330152109, 0.6768198921084811, 0.6419281254897065, 0.6703171990103243, 0.6512883355556676, 0.6587351597836105, 0.6522427447239478], [0.49540025238001784, 0.5488547113461075, 0.6101829407462287, 0.6392781882489837, 0.6497007749049502, 0.6413969761472249, 0.6714171997700442, 0.6546908067990413, 0.6659482557438382, 0.63698787053006

In [13]:
print(overall_list_dev_recall)

[[0.518741782180146, 0.6220680603972358, 0.6055386897620354, 0.626695668314559, 0.6474177349445484, 0.6397481919632656, 0.6428413807976571, 0.6569916302039464, 0.6418201398880925, 0.6542795752709437], [0.5566623535340708, 0.6155899967795975, 0.6086521766493985, 0.6346322025313621, 0.6379873718135273, 0.674237982489033, 0.6312954264295277, 0.6363834988625249, 0.6460541212405707, 0.6576142353794667], [0.5369401479614052, 0.5917973177033051, 0.5994191766158677, 0.6031490314197081, 0.626289051022507, 0.6293242858194069, 0.6368362923856206, 0.6188124323884565, 0.6255535435327978, 0.6282836191156623], [0.5373995236935101, 0.6105540448374788, 0.6375283535847167, 0.6714669449779055, 0.632587090780368, 0.6466782351742684, 0.6679151264964027, 0.6629669975507687, 0.6548841917303053, 0.6505840194469398], [0.5368601301243108, 0.5732319660090143, 0.6273803927318687, 0.6276597890260261, 0.6417106089997648, 0.6207633042980719, 0.6511672808040218, 0.6428394608863358, 0.6414834821104864, 0.6299972577900

In [14]:
print(overall_list_dev_f1score)

[[0.5008894693158956, 0.5987011097510446, 0.5888775084816721, 0.6031982194628654, 0.6350361999859736, 0.6313816597061915, 0.6276632799767761, 0.6306550399810231, 0.616464889714759, 0.6360438003262413], [0.5262394051012577, 0.5887560699803492, 0.5963572936637273, 0.610319994452159, 0.6176059579138096, 0.6408464094443331, 0.6084409750213965, 0.6109255306357086, 0.6225142981959425, 0.6402853433472885], [0.4931755744839374, 0.5710127946554583, 0.5810657670399196, 0.5924234020831952, 0.6076941771082, 0.5990251226160755, 0.6158666200037338, 0.6119518524279798, 0.6085618534268428, 0.6232894241912684], [0.5115207996179639, 0.5892529969465394, 0.6254637395896115, 0.6410841648218444, 0.6330893986092397, 0.6241874816521584, 0.6467490168281513, 0.6372009850259417, 0.6353026569790602, 0.6300630224593353], [0.4984768700163665, 0.5419531895322874, 0.59716191784685, 0.614923894503853, 0.6257615457877278, 0.6106119103697728, 0.6363418407487581, 0.6256740603010023, 0.6299706949122605, 0.6128156725159701

In [15]:
#The best model is 3